In [ ]:
pip install pytube

In [1]:
from pytube import YouTube

In [2]:
!conda list | grep pytube

pytube                    11.0.1                   pypi_0    pypi


In [3]:
# misc
import os
import shutil
import math
import datetime
# plots
import matplotlib.pyplot as plt
%matplotlib inline
# image operation
import cv2

In [5]:
video_url = 'https://www.youtube.com/watch?v=YWhSQpUNGgY' 
# video.streams.filter(file_extension = "mp4").all()
youtube = YouTube(video_url)
video = youtube.streams.first()
video.download(filename="Sticks_and_Stones.mp4")
# video.filesize

'/Users/colbyr/Documents/GitHub/downloader/Sticks_and_Stones.mp4'

In [ ]:
import requests
file_url = "https://www.youtube.com/watch?v=YWhSQpUNGgY"
file_name = "Sticks and Stone.mp4"

r = requests.get(file_url, stream = True)

# download started 
with open(file_name, 'wb') as f: 
    for chunk in r.iter_content(chunk_size = 1024*1024): 
        if chunk: 
            f.write(chunk) 
  
print( "%s downloaded!\n"%file_name )

Sticks and Stone.mp4 downloaded!



In [6]:
from pytube import YouTube

In [13]:
youtube_video_url = 'https://youtu.be/bIjCNQT5BNU'
track_title = '8hr Binarual Sleep.mp3'
 
yt_obj = YouTube(youtube_video_url)

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
audio = yt_obj.streams.filter(only_audio=True).first()
audio.download('./downloads', filename=track_title)

'/Users/colbyr/Documents/GitHub/downloader/./downloads/8hr Binarual Sleep.mp3'